# Azure AI Search cu integrarea NVIDIA NIM și LlamaIndex

În acest notebook, vom demonstra cum să utilizăm modelele AI de la NVIDIA și LlamaIndex pentru a crea un pipeline puternic de Generare Augmentată prin Recuperare (RAG). Vom folosi LLM-urile și embedding-urile NVIDIA, le vom integra cu Azure AI Search ca depozit vectorial și vom efectua RAG pentru a îmbunătăți calitatea și eficiența căutării.

## Beneficii
- **Scalabilitate**: Utilizați modelele lingvistice mari de la NVIDIA și Azure AI Search pentru recuperare scalabilă și eficientă.
- **Eficiență economică**: Optimizați căutarea și recuperarea cu stocare vectorială eficientă și tehnici de căutare hibridă.
- **Performanță ridicată**: Combinați LLM-uri puternice cu căutarea vectorială pentru răspunsuri mai rapide și mai precise.
- **Calitate**: Mențineți o calitate ridicată a căutării prin ancorarea răspunsurilor LLM cu documente relevante recuperate.

## Cerințe preliminare
- 🐍 Python 3.9 sau mai recent
- 🔗 [Serviciul Azure AI Search](https://learn.microsoft.com/azure/search/)
- 🔗 Cheie API NVIDIA pentru acces la LLM-urile și embedding-urile NVIDIA prin microserviciile NVIDIA NIM

## Funcționalități acoperite
- ✅ Integrarea LLM NVIDIA (vom folosi [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ Embedding-uri NVIDIA (vom folosi [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ Moduri avansate de recuperare Azure AI Search
- ✅ Indexarea documentelor cu LlamaIndex
- ✅ RAG folosind Azure AI Search și LlamaIndex cu LLM-urile NVIDIA

Să începem!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## Instalare și Cerințe  
Creează un mediu Python utilizând versiunea Python >3.10.

## Să Începem!  


Pentru a începe, ai nevoie de un `NVIDIA_API_KEY` pentru a utiliza modelele NVIDIA AI Foundation:
1) Creează un cont gratuit cu [NVIDIA](https://build.nvidia.com/explore/discover).
2) Fă clic pe modelul dorit.
3) Sub secțiunea Input, selectează fila Python, apoi fă clic pe **Get API Key** și apoi pe **Generate Key**.
4) Copiază și salvează cheia generată ca NVIDIA_API_KEY. De aici, ar trebui să ai acces la endpoint-uri.


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## Exemplu RAG folosind LLM și Embedding
### 1) Inițializați LLM-ul
`llama-index-llms-nvidia`, cunoscut și sub numele de conectorul LLM de la NVIDIA, vă permite să vă conectați și să generați din modelele compatibile disponibile în catalogul API NVIDIA. Consultați aici lista modelelor de completare chat: https://build.nvidia.com/search?term=Text-to-Text

Aici vom folosi **mixtral-8x7b-instruct-v0.1**


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) Inițializează Embedding-ul  
`llama-index-embeddings-nvidia`, cunoscut și sub numele de conectorul Embeddings de la NVIDIA, îți permite să te conectezi și să generezi din modelele compatibile disponibile în catalogul API NVIDIA. Am selectat `nvidia/nv-embedqa-e5-v5` ca model de embedding. Vezi aici o listă de modele pentru embedding de text: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding  


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) Creează un motor de interogare pentru a pune întrebări despre datele tale

Iată o interogare folosind căutarea pură pe vectori în Azure AI Search și fundamentarea răspunsului pentru LLM-ul nostru (Phi-3.5-MOE)


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

Iată o interogare utilizând căutarea hibridă în Azure AI Search.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### Analiza căutării vectoriale
Răspunsul LLM surprinde cu acuratețe principalele consecințe economice menționate în textul sursă referitor la piața de capital din Rusia. Mai exact, se afirmă că piața de capital din Rusia a suferit o scădere semnificativă, pierzând 40% din valoarea sa, iar tranzacționarea a fost suspendată din cauza situației în desfășurare. Acest răspuns se aliniază bine cu informațiile furnizate în sursă, indicând faptul că LLM a identificat și rezumat corect detaliile relevante privind impactul asupra pieței de capital ca urmare a acțiunilor Rusiei și sancțiunilor impuse.

#### Comentariu asupra nodurilor sursă
Nodurile sursă oferă o relatare detaliată a consecințelor economice pe care Rusia le-a întâmpinat din cauza sancțiunilor internaționale. Textul subliniază că piața de capital din Rusia a pierdut 40% din valoarea sa, iar tranzacționarea a fost suspendată. În plus, menționează alte repercusiuni economice, cum ar fi devalorizarea Rublei și izolarea mai amplă a economiei Rusiei. Răspunsul LLM a distilat eficient punctele esențiale din aceste noduri, concentrându-se pe impactul asupra pieței de capital, conform cerinței interogării.


Acum, să analizăm o interogare în care Căutarea Hibridă nu oferă un răspuns bine fundamentat:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### Căutare Hibridă: Analiza Răspunsului LLM
Răspunsul LLM din exemplul de Căutare Hibridă indică faptul că contextul furnizat nu specifică data exactă a invaziei Rusiei în Ucraina. Acest răspuns sugerează că LLM utilizează informațiile disponibile în documentele sursă, dar recunoaște lipsa detaliilor precise în text.

Răspunsul este corect în identificarea faptului că contextul menționează evenimente legate de agresiunea Rusiei, dar nu indică data exactă a invaziei. Acest lucru demonstrează capacitatea LLM de a înțelege informațiile furnizate, în timp ce recunoaște lacunele din conținut. LLM sugerează eficient utilizatorului să caute surse externe sau înregistrări istorice pentru data exactă, afișând un nivel de precauție atunci când informațiile sunt incomplete.

### Analiza Nodurilor Sursă
Nodurile sursă din exemplul de Căutare Hibridă conțin fragmente dintr-un discurs care discută răspunsul SUA la acțiunile Rusiei în Ucraina. Aceste noduri subliniază impactul geopolitic mai larg și măsurile luate de SUA și aliații săi ca răspuns la invazie, dar nu menționează data specifică a invaziei. Acest lucru este în concordanță cu răspunsul LLM, care identifică corect faptul că contextul nu conține informații precise despre data invaziei.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### Analiza hibridă cu reranking: Răspunsul LLM
În exemplul de hibrid cu reranking, răspunsul LLM oferă context suplimentar, menționând că evenimentul a avut loc cu șase zile înainte de discurs. Acest lucru indică faptul că LLM-ul poate deduce data invaziei pe baza momentului discursului, chiar dacă încă necesită cunoașterea exactă a datei discursului pentru precizie.

Acest răspuns demonstrează o capacitate îmbunătățită de a utiliza indicii contextuale pentru a oferi un răspuns mai informativ. Subliniază avantajul reranking-ului, prin care LLM-ul poate accesa și prioritiza informații mai relevante pentru a oferi o aproximare mai apropiată a detaliului dorit (adică data invaziei).

### Analiza nodurilor sursă
Nodurile sursă din acest exemplu includ referințe la momentul invaziei Rusiei, menționând în mod specific că aceasta a avut loc cu șase zile înainte de discurs. Deși data exactă nu este încă menționată explicit, nodurile oferă un context temporal care permite LLM-ului să ofere un răspuns mai nuanțat. Includerea acestui detaliu evidențiază modul în care reranking-ul poate îmbunătăți capacitatea LLM-ului de a extrage și deduce informații din contextul furnizat, rezultând un răspuns mai precis și mai informativ.


**Notă:**
În acest notebook, am utilizat microservicii NVIDIA NIM din Catalogul de API-uri NVIDIA.  
API-urile menționate mai sus, `NVIDIA (llms)`, `NVIDIAEmbedding`, și [Azure AI Search Semantic Hybrid Retrieval (built-in reranking)](https://learn.microsoft.com/azure/search/semantic-search-overview). Rețineți că API-urile de mai sus pot susține și microservicii găzduite local.

**Exemplu:**
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")```



---

**Declinarea responsabilității**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să rețineți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa natală ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm răspunderea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
